<center>Copyright 2020 Parkland Health & Hospital System </center>

This program entitled “Parkland Trauma Index of Mortality” is free software and is distributed under the terms of the GNU Lesser General Public License (LGPL). You can redistribute it and/or modify it under the terms of the GNU LGPL as published by the Free Software Foundation, either version 3 of the License or any later version. This program is distributed WITHOUT ANY WARRANTY; without even THE IMPLIED WARRANTY OF MERCHANTABILITY or FITTNESS FOR A PARTICULAR PURPOSE. See the GNU LGPL for more details. You should have received a copy of the GNU LGPL along with this program; if not, see https://www.gnu.org/licenses.



## Quick Start

- Point global variables to raw data path. 
- 1_Trauma_Data_Preparation prepares the data for analysis
- 2_Trauma_Model applies some final cleaning steps and fits models


________________________________________________________
## Methodology 
**Out-of-ICU model validation:** Generate misclassification table, ROC curve, C-statistic

**Predict mortality:** Classification model comparison

**Unit of analysis:** [12] hour groupings of an inpatient stay associated with a trauma activation, with optional resampling to test for bias, and optimize time-to-intervene.

**Outcome:** mortality within hospital stay
________________________________________________________
## Dependencies 
### Data Sources and Connections
- Clinical + Demographics: Connection to Parkland's Clarity and EDW data via Composite ODBC

### Software
- Python 3.6 
- SQLite

### Libraries
- Pandas
- NumPy
- CSV
- SQLite3
- Matplotlib
- Seaborn
- FancyImpute (mice.py - local import)
- MissingNo


In [ ]:
# Load Dependencies
from time import time
import pandas as pd
import csv
import re
import numpy as np
import sqlite3 as lite
import missingno as mn
import seaborn as sns
import collections
from matplotlib import pyplot as plt

sns.set(style="white", color_codes=True)
%matplotlib inline

In [ ]:
# Global variables
## Set number of hours to use when resampling clinical data
TIME_BINS = '12H'

## Set maximum number of time periods to include per encounter
MAX_PERIODS = 60

## Set number of jobs to use in parallel (-1 = max)
N_JOBS = 3

## Set base path for raw data

DATA_PATH = 'path to data file'


## Set default SQLite database file; default will be in same path as raw data files
SQLITE_DB = DATA_PATH + "trauma_mortality_backup_tast.db"

In [ ]:
## Replace spaces in column names (caused by IDT/WebBI) with underscores; make all headers lowercase
### Inputs: list of dataframes, list of dataframe names
### Outputs: None. Makes changes in place.
def ColumnSpaceToUnderscore(all_data, file_list):   
    i = 0
    while i < len(file_list):
        column_dict = {}
        for j in range(0, len(all_data[file_list[i]].columns.values)):
            column_dict[all_data[file_list[i]].columns.values[j]] = all_data[file_list[i]].columns.values[j].replace(' ', '_')
        all_data[file_list[i]].rename(columns=column_dict, inplace=True)
        all_data[file_list[i]].rename(columns=str.lower, inplace=True)
        i += 1
    print('Column names are now lower-case and free of spaces.')

In [ ]:
## Confirm correct datatypes coming across in all input fields
def CheckDataTypes(list_of_dataframes, file_list):
    i = 0
    while i < len(file_list):
        print("  Table:", file_list[i])
        print(all_data[file_list[i]].dtypes, "\n")
        i += 1

In [ ]:
## Write selected dataframe to default SQLite database, replacing if found
def DataFrameToSQL(df, df_name):
    connection = lite.connect(SQLITE_DB)
    with connection:
        df.to_sql(df_name, connection, schema = None, if_exists='replace', index=False)
        print('%s backed up to default SQLite database.' % df_name)

In [ ]:
## Retrieve SQL table from default SQLite database and store as a dataframe
def SQLToDataFrame(table_name, date_columns=[]):
    connection = lite.connect(SQLITE_DB)
    sql = 'select * from ' + table_name
    with connection:
        df = pd.read_sql(sql, connection, parse_dates=date_columns)
    return(df)

## Setup Data Ingestion

In [ ]:
## List of input file names 
file_list = ['admission', 'dx', 'flowsheet', 'lab', 'patient', 'service_team', 'rx','registry']

## Dict of input files' date columns (by index) to convert 
date_columns = {'admission': [2,3],  'dx': [2], 'flowsheet': [4], 
                'lab': [4, 5], 'patient': [7, 8], 'service_team': [], 'rx': [3],'registry' : [1]}

## Initialize empty OrderedDict to contain working dataframes. 
all_data = collections.OrderedDict()

## Read tsv files into Pandas dataframes, and store in dict
### Note: parse_dates is quite slow compared to pd.to_datetime
for file in file_list:
    file_path = DATA_PATH + file + '.txt'
    df = pd.read_csv(file_path, sep = '\t', header = 0, thousands = ',',
                     parse_dates = date_columns[file], infer_datetime_format=True)
    all_data[file] = df
    print(file, df.shape)

In [ ]:
ColumnSpaceToUnderscore(all_data, file_list)

In [ ]:
CheckDataTypes(all_data, file_list)

In [ ]:
## Store tables in temporary SQLite db
## Initialize SQLite database for temporary storage
connection = lite.connect(SQLITE_DB)

### /with/ will automatically close connection when complete

with connection:
    i = 0
    while i < len(file_list):
        all_data[file_list[i]].to_sql(file_list[i], 
                                      connection, 
                                      schema = None, 
                                      if_exists='replace',
                                      index=False)
        print(file_list[i], 'table backed up to SQLite.')
        i+=1
        
    connection.commit()

## Data Ingestion

In [ ]:
## Distinct patients/encounters for each data set
for each in file_list:
    if each == 'flowsheet':
        print(each, all_data[each].patientkey.nunique())
    elif each == 'service_team':
        print(each, all_data[each].encounterepiccsn.nunique())
    elif each in ('admission'):
        print(each, all_data[each].pat_enc_csn_id.nunique())
   
    else:
        try:
            print(each, all_data[each].patientkey.nunique())
        except:
            pass
    

In [ ]:
## Assign new data frame names to all input data sets
admission, dx, flowsheet, lab, patient, service_team, rx, registry = iter(all_data.values())
del all_data

In [ ]:
## Extract date from admission for joining 
admission['adm_date'] = admission['hosp_admsn_time'].dt.date
admission['adm_date'] = pd.to_datetime(admission['adm_date'])

### Pivots and Resampling

In [ ]:
# Utility function: downsample dates with hierarchical index
def DateGrouper(df, num_levels=3):
    level_values = df.index.get_level_values
    print(level_values)
    # Downsampling method not defined; make sure to chain with .sum(), .last(), etc.
    base_levels = list(range(0, num_levels-1))
    return (df.groupby([level_values(i) for i in base_levels]
                       +[pd.Grouper(freq=TIME_BINS, level=-1)]))

In [ ]:
# Utility function: pivot long results into wide format
### Inputs: dataframe, hierarchical index as list, date column, field name column, field value column, TIME_BINS
### Output: pivoted dataframe
def LongToWide(df, index_fields, date_field, name_field, value_field):
    df_temp = df.copy()
    
    full_index = index_fields
    full_index.append(name_field)
    index_fields.append(date_field)
    #df_temp.set_index(full_index, inplace=True, drop=False)
    
    ## Replace spaces in name_field with underscores
    df_temp[name_field].replace(regex=True, inplace=True, to_replace=r'[ ]', value=r'_')
    ## Remove commas in name_field 
    df_temp[name_field].replace(regex=True, inplace=True, to_replace=r'[,]', value=r'')
    
    #print(index_fields)
    
     ## Use DateGrouper to resample to TIME_BINS global var, using last observation
    #df_temp = DateGrouper(df_temp, num_levels=len(index_fields)).last()

    index_fields.remove(name_field)
    
    df_pivot = pd.pivot_table(df_temp, 
                             index=index_fields,
                             values = value_field,
                             columns = name_field,
                             aggfunc = max)#np.max)#np.mean)#np.nanmax) 
    
    ## Use DateGrouper to resample to TIME_BINS global var, using last observation
    #df_pivot = DateGrouper(df_pivot, num_levels=len(index_fields)).last()
    
    df_pivot.reset_index(inplace=True)
        
    # Convert all columns to lower-case
    df_pivot.rename(columns=str.lower, inplace=True)
    
    return(df_pivot)

In [ ]:
 ## Remove rows with missing measure values
flow_na_removed = flowsheet.dropna(axis=0, how='any', subset=['flowvalue'], inplace=False)

## Replace spaces in flowname with underscores
flow_na_removed['flowname'].replace(regex=True, inplace=True, to_replace=r'[ ]', value=r'_')

## Remove commas in flowname 
flow_na_removed['flowname'].replace(regex=True, inplace=True, to_replace=r'[,]', value=r'')

flow_numerical_columns = ['PULSE_OXIMETRY', 'TEMPERATURE', 'PULSE',
                          'BLOOD_PRESSURE', 'RESPIRATIONS', 'PKMOD_R_CPN_GLASGOW_COMA_SCALE_SCORE']

flow_cat_columns = []

flow_all_columns = flow_numerical_columns + flow_cat_columns

In [ ]:
%%time

flow_pivot = LongToWide(df=flow_na_removed[flow_na_removed.flowname.isin(flow_all_columns)], 
                        index_fields=['patientkey', 'encounterkey'], 
                        date_field='flowdatetime', 
                        name_field='flowname', 
                        value_field='flowvalue')

In [ ]:
# Select blood pressure measures, convert to strings, split on "/" character, convert to dictionary
flow_pivot = flow_pivot.set_index(['patientkey','encounterkey','flowdatetime'])
bp_split = flow_pivot.blood_pressure.str.split('/').to_dict()


In [ ]:
# Create dataframe from blood pressure dict, transpose, rename columns
bp_df = pd.DataFrame.from_dict(bp_split, orient='columns').T.rename(columns = {0: 'systolic', 1: 'diastolic'}) 

# Merge bp dataframe back into pivoted flowsheet dataframe
flow_temp = flow_pivot.reset_index().join(bp_df, how="left", on=['patientkey','encounterkey','flowdatetime'])#pd.merge(flow_pivot2, bp_df, how="left", left_index=True, right_index=True)

flow_pivot = flow_temp

In [ ]:
mn.matrix(flow_pivot)

In [ ]:
lab = lab.replace({'labcommonname': {'POC HEMOGLOBIN': 'HEMOGLOBIN', 'GLUCOSE RANDOM': 'GLUCOSE',\
                                       'GLUCOSE UA': 'GLUCOSE', 'GLUCOSEPOC': 'GLUCOSE', 'POC POTASSIUM': 'POTASSIUM'}})

In [ ]:
%%time
labs_pivot = LongToWide(df=lab, 
                        index_fields=['patientkey', 'encounterkey'], 
                        date_field='labcollectiontime', 
                        name_field='labcommonname', 
                        value_field='labvalue')

In [ ]:
%%time
labs_flag_pivot_base = LongToWide(df=lab, 
                                  index_fields=['patientkey', 'encounterkey'], 
                                  date_field='labcollectiontime', 
                                  name_field='labcommonname', 
                                  value_field='labflag')

In [ ]:
## Add _flag suffix to all columns except index columns.
labs_flag_pivot = labs_flag_pivot_base.drop(['patientkey', 'encounterkey', 'labcollectiontime'],axis=1)\
                                      .add_suffix('_flag')\
                                      .join(labs_flag_pivot_base[['patientkey','encounterkey','labcollectiontime']])

In [ ]:
## Join lab values and lab flag pivots on patient, encounter, and time
## OFF due to no flags in physiologic model
#labs_pivot = pd.merge(labs_pivot, labs_flag_pivot, on=['patientkey','encounterkey','labcollectiontime'], how='outer')

In [ ]:
## Outer join labs and flowsheets on patient, encounter, and time
labs_flow_pivot = pd.merge(labs_pivot, flow_pivot, left_on=['patientkey','encounterkey','labcollectiontime'], 
                           right_on=['patientkey','encounterkey','flowdatetime'], how='outer')

In [ ]:
## Merge different versions of patient and encounter keys to reduce confusion
labs_flow_pivot['patientkey'] = labs_flow_pivot['patientkey'].fillna(labs_flow_pivot.patientkey)
labs_flow_pivot['encounterkey'] = labs_flow_pivot['encounterkey'].fillna(labs_flow_pivot.encounterkey)
#labs_flow_pivot.drop(['patientkey','encounterkey'], axis=1, inplace=True)
labs_flow_pivot.head()

In [ ]:
DataFrameToSQL(labs_flow_pivot, 'labs_flow_pivot')

In [ ]:
labs_flow_pivot = SQLToDataFrame(table_name='labs_flow_pivot', date_columns=['labcollectiontime', 'flowdatetime'])

## Data Processing

In [ ]:
## Convert analgesic dosages to morphine-equivalent-doses
#rx['dose_mde'] = rx.conversion_to_oral_morphine * rx.rx_dose_min

## Re-index and select columns to include
rx = rx[['patientkey','encounterepiccsn', 'encounterkey', 'rxadministrationinstant', 'rxdosemin', 'rxshortname', 'rxroutetype']]
#rx.set_index(['patient_key', 'encounter_key', 'rx_administration_instant'], inplace=True)

In [ ]:
labs_flow_pivot['current_datetime'] = labs_flow_pivot['flowdatetime'].fillna(labs_flow_pivot['labcollectiontime'])

In [ ]:
merge_rx_flow = pd.merge(rx, labs_flow_pivot,
                         left_on=['patientkey', 'encounterkey', 'rxadministrationinstant'],
                         right_on=['patientkey', 'encounterkey', 'current_datetime'],
                         how='outer')

In [ ]:
## Create coalesced time elapsed since first measurement
labs_flow_pivot2 = merge_rx_flow.copy()
labs_flow_pivot2['current_datetime'] = labs_flow_pivot2['flowdatetime'].fillna(labs_flow_pivot2['labcollectiontime']).fillna(merge_rx_flow['rxadministrationinstant'])

labs_flow_pivot2 = labs_flow_pivot2.join(labs_flow_pivot2.groupby(['patientkey','encounterkey']).current_datetime.min(), on=['patientkey','encounterkey'], rsuffix='_min')
labs_flow_pivot2['time_elapsed'] = (labs_flow_pivot2.current_datetime - labs_flow_pivot2.current_datetime_min)#.astype('m8[h]')
#labs_flow_pivot2.head()

In [ ]:
labs_flow_pivot2.drop(['labcollectiontime', 'rxadministrationinstant', 'flowdatetime', 'blood_pressure'], axis=1, inplace=True)

In [ ]:
%%time

rx_flow_labs = labs_flow_pivot2.copy().set_index(['patientkey','encounterkey','time_elapsed'], drop=True)
rx_sum = DateGrouper(rx_flow_labs[['rxdosemin']].copy()).sum().fillna(0).reset_index()

In [ ]:
rx_sum['time_elapsed'] = rx_sum['time_elapsed'].astype('timedelta64[h]')

In [ ]:
rx_sum['time_sequence'] = rx_sum.time_elapsed.astype('timedelta64[h]') / np.timedelta64(12, 'h')

In [ ]:
DataFrameToSQL(rx_sum, 'rx_sum')

In [ ]:
rx_sum = SQLToDataFrame(table_name='rx_sum')
# rx_sum['time_elapsed'] = rx_sum.time_sequence * np.timedelta64(12, 'h')

### Max/Min/Velocity

In [ ]:
# time_vars = ['albumin', 'alt','ast', 'base_exc_art', 'bilirubin_total', 'co2', 'creatinine', 'crp', 'glucosepoc', 'glucose_random', \
#              'glucose_ua', 'hco3_art', 'hco3_ven', 'inr', 'lactate', 'lactate_bld_arterial', 'platelets','poc_inr', 'prealbumin',\
#              'sed_rate', 'wbc', 'pulse', 'pulse_oximetry','respirations', 'temperature', 'systolic', 'diastolic']

# labs = ['albumin', 'alt', 'ast', 'base_exc_art', 'bilirubin_total', 'co2', 'creatinine', 'crp', 'glucosepoc', 'glucose_random',\
#         'glucose_ua', 'hco3_art', 'hco3_ven', 'inr', 'lactate', 'lactate_bld_arterial', 'platelets', 'poc_inr', 'prealbumin', \
#         'sed_rate', 'wbc']

# vitals = list(set(time_vars) - set(labs))

labs = ['platelets', 'wbc', 'lactate', 'lactate_bld_arterial', 'base_exc_art', 'albumin', 'prealbumin', 'sed_rate', 'glucose', \
        'hco3_ven', 'hco3_art', 'inr', 'poc_inr', 'creatinine', 'bilirubin_total', 'ast', 'alt', 'crp', 'co2', 'potassium', 'hemoglobin']
vitals = ['temperature', 'pulse', 'pulse_oximetry', 'systolic']
time_vars = labs + vitals

max_vars = ['temperature', 'pulse', 'wbc', 'lactate', 'inr', 'creatinine', 'ast', 'alt', 'bilirubin_total', 'systolic', 'potassium']
min_vars = ['temperature', 'pulse', 'pulse_oximetry', 'platelets', 'base_exc_art', 'albumin', 'systolic', 'potassium', 'hemoglobin']
avg_vars = ['pulse', 'pulse_oximetry', 'systolic']

In [ ]:
# ## Recode all labs and vitals as numeric
# for col in time_vars:
#     ## Use regex to search for symbols, text, and spaces; replace them with empty string; convert to numeric
#     try:
#         rx_flow_labs[col] = pd.to_numeric(re.sub(r"[<>A-z ]", "", rx_flow_labs[col]),errors='coerce')
#     except:
#         pass

## Recode all labs and vitals as numeric
for col in time_vars:
    ## Use regex to search for symbols, text, and spaces; replace them with empty string; convert to numeric
    if col in rx_flow_labs.columns.values:
        try:
            rx_flow_labs[col] = pd.to_numeric(rx_flow_labs[col].astype(str).str.replace(r"[<>A-z = ,]", ""))
        except TypeError:
            print(col, 'could not be converted!')
            pass

In [ ]:
%%time 
flow_labs_to_fill = rx_flow_labs.copy().drop(['rxdosemin'],axis=1) #, 'rx_short_name', 'rx_route_type'
flow_labs_filled = flow_labs_to_fill.copy().groupby(level=[0,1]).fillna(method='ffill', downcast='infer').groupby(level=[0,1]).fillna(method='bfill', downcast='infer')

In [ ]:
%%time 
flow_labs_last = DateGrouper(flow_labs_filled).last()

In [ ]:
%%time
timevars_max = DateGrouper(flow_labs_filled.copy()[max_vars]).max()
timevars_max = timevars_max.add_suffix('_max')

timevars_min = DateGrouper(flow_labs_filled.copy()[min_vars]).min()
timevars_min = timevars_min.add_suffix('_min')

timevars_avg = DateGrouper(flow_labs_filled.copy()[avg_vars]).mean()
timevars_avg = timevars_avg.add_suffix('_avg')


## Calculate within-period max-min difference
#itals_max = DateGrouper(flow_labs_filled.copy()[vitals]).max()

In [ ]:
# %%time

# vitals_min = DateGrouper(flow_labs_filled.copy()[vitals]).min()
# vitals_diff = vitals_max.astype(float) - vitals_min.astype(float)
# vitals_diff = vitals_diff.add_suffix('_diff')
# #vitals_diff.sort_index().head()

In [ ]:
rx_sum['time_elapsed'] = rx_sum.time_sequence * np.timedelta64(12, 'h')
rx_sum.set_index(keys=['patientkey', 'encounterkey', 'time_elapsed'], inplace=True)

In [ ]:
rx_flow_labs = pd.merge(rx_sum, flow_labs_last, left_index=True, right_index=True, how='outer')
rx_flow_labs = pd.merge(rx_flow_labs, timevars_max, left_index=True, right_index=True, how='outer')
rx_flow_labs = pd.merge(rx_flow_labs, timevars_min, left_index=True, right_index=True, how='outer')
rx_flow_labs = pd.merge(rx_flow_labs, timevars_avg, left_index=True, right_index=True, how='outer')
rx_flow_labs.sort_index(level=1).head()

# rx_flow_labs = pd.merge(rx_sum, flow_labs_last, left_index=True, right_index=True, how='outer')
# rx_flow_labs = pd.merge(rx_flow_labs, vitals_diff, left_index=True, right_index=True, how='left')
# rx_flow_labs.sort_index(level=1).head()

In [ ]:
#Sort data ascending by nullity
sorted_data = mn.nullity_sort(rx_flow_labs, sort='ascending')
mn.bar(sorted_data)

In [ ]:
rx_flow_labs2 = rx_flow_labs.copy().reset_index()

## SQLite mangles timedelta fields, so create a durable time_sequence that is just a counter for observation windows
rx_flow_labs2['time_sequence'] = rx_flow_labs2.time_elapsed / np.timedelta64(12, 'h')
DataFrameToSQL(rx_flow_labs2.drop('time_elapsed',axis=1), 'rx_flow_labs')

In [ ]:
rx_flow_labs = SQLToDataFrame(table_name='rx_flow_labs', date_columns=['current_datetime', 'current_datetime_min'])

# ## Reconvert durable time sequence into time elapsed (timedelta)
# rx_flow_labs['time_elapsed'] = rx_flow_labs.time_sequence * np.timedelta64(12, 'h')
# rx_flow_labs.set_index(['patientkey', 'encounterkey', 'time_elapsed'], inplace=True)

## Merging

In [ ]:
# def DataFrameToSQL1(df, df_name):
#     connection = lite.connect(DATA_PATH+'trauma_tast_merge_temp1.db')
#     with connection:
#         df.to_sql(df_name, connection, schema = None, if_exists='replace', index=False)
#         print('%s backed up to default SQLite database.' % df_name)

In [ ]:
# DataFrameToSQL1(merge_temp1,'merge_temp1')

In [ ]:
#Left join ADMISSION with RX_FLOW_LABS
merge_temp1 = pd.merge(admission,rx_flow_labs.reset_index(), left_on = ['pat_enc_csn_id'], right_on = 'encounterepiccsn', how ='left')

In [ ]:
## Left join merge_temp1 and patient on patientkey
merge_temp1 = pd.merge(merge_temp1, patient, 
                                   left_on='patientkey', 
                                   right_on='patientkey', 
                                   how='left')

In [ ]:
service_team_sicu = service_team[service_team['serviceteamname'].isin(['SICU TEAM', 'NCC TEAM', 'NEURO CRITICAL CARE'])]
service_team_sicu = service_team_sicu.drop_duplicates(keep = 'first')

In [ ]:
# Left join merge_temp1 and service_team where service team name in SICU team, NCC team, Neuro Critical care team
merge_temp1 = pd.merge(merge_temp1,service_team_sicu, left_on='pat_enc_csn_id', right_on = 'encounterepiccsn',how = 'left')

In [ ]:
## Remove registry rows with invalid identifers
registry_clean = registry[registry.mrn!='0 0 0 0 0 0 0 0']
registry_clean = registry_clean[registry_clean.mrn!='                    '] 

## Remove registry rows with missing identifiers
registry_clean = registry_clean[pd.isnull(registry_clean.mrn)==False]

## Remove invalid character from registry identifier field
registry_clean.mrn.replace(regex=True, inplace=True, to_replace=r'[`]', value='')

## Convert registry identifier to float
registry_clean.mrn = registry_clean.mrn.astype(float)

## Remove invalid characters from ISS variable
registry_clean.iss = registry_clean.iss[~registry_clean.iss.isin(['   ','UA'])]

## Convert date of arrival to datetime
registry_clean['Date of Arrival'] = pd.to_datetime(registry_clean['date_of_arrival'])

In [ ]:
## Left join merge_temp1 and registry_clean on ['mrn', 'date_of_arrival']
merge_temp1 = pd.merge(merge_temp1, registry_clean, 
                                   left_on=['primarymrn', 'adm_date'],
                                   right_on=['mrn', 'date_of_arrival'], 
                                   how='left')

In [ ]:
#Adding a sicu flag in case we need to subset only SICU patients from the cohort
merge_temp1['sicu_flag'] = 0
for team in ['SICU TEAM', 'NCC TEAM', 'NEURO CRITICAL CARE']:
    #df.ix[(df['A'] == df['B']), 'C'] = 0
    merge_temp1.loc[merge_temp1['serviceteamname']==team,'sicu_flag'] = 1

In [ ]:
## Left join merge_temp1 and dx on ['patientkey', 'encounterkey']
merge_temp1_dx =  pd.merge(merge_temp1, dx, 
                    on=['patientkey', 'encounterkey'],
                     how='left')

In [ ]:
#print(merge_temp1_dx.shape)
no_burn_id = merge_temp1_dx[merge_temp1_dx.dxname.str.contains('burn|Burn')==False]['pat_enc_csn_id'].unique()


In [ ]:
## Including only non-burn patients
merge_temp1_dx = merge_temp1_dx[merge_temp1_dx.pat_enc_csn_id.isin(no_burn_id)].sort_values('pat_enc_csn_id')

In [ ]:
merge_temp1 =  merge_temp1_dx.drop(['dxstartdate', 'hospitaldiagnosis',
       'emergencydepartmentdiagnosis', 'dxname', 'dxdisplayname', 'dxtype',
       'dxstatus', 'dxpresentonadmission', 'dxcodetype', 'dxcode'], axis = 1).drop_duplicates(keep = 'first')

In [ ]:
# Deleting few dataframes to combat memory error
del labs_flow_pivot
del rx_sum
del rx_flow_labs

In [ ]:
DataFrameToSQL(merge_temp1_dx,'merge_temp1_dx')

In [ ]:
DataFrameToSQL(merge_temp1, 'merge_temp1')

In [ ]:
merge_all = merge_temp1.copy()

In [ ]:
merge_all['time_elapsed'] = merge_all.time_sequence * np.timedelta64(12, 'h')
merge_all['time_elapsed'].fillna(0, inplace=True)
merge_all['time_sequence'] = merge_all.time_elapsed / np.timedelta64(12, 'h')
merge_all['hours_elapsed'] = merge_all.time_elapsed / np.timedelta64(1, 'h')

In [ ]:
## Drop unneeded or highly missing columns
try:
    merge_all = merge_all.drop(['department_name', 'encounterepiccsn_x','encounterepiccsn_y', 'religion',
                            'hsp_account_id', 'patientkey', 'encounterkey'], axis=1)
except:
    pass

In [ ]:
## Calculate BMI
#merge_all['bmi'] = ((merge_all['weight_lb'].astype(float)) / merge_all['height_in'].astype(float)**2) * 703
## Replace BMI's beyond 7:100 with null, to be imputed later due to bad weight/height measurements
#merge_all['bmi'] = merge_all.bmi.where(merge_all.bmi>7, None).where(merge_all.bmi < 100, None).astype(float)

## Calculate age in years
merge_all['age'] = (merge_all.hosp_admsn_time - merge_all.birthdate).astype('<m8[Y]') 

## Create death flags for potential dependent variables
merge_all['death_flag_ever'] = abs((merge_all['deathdate'].isnull()).astype(float)-1)
merge_all['death_flag_inhospital'] = (merge_all['deathdate'] <= merge_all['hosp_dischrg_time']).astype(float)
merge_all['death_flag_registry'] = pd.get_dummies(merge_all['death_case'])['Yes']
merge_all['death_flag_this_period'] = ((merge_all['time_elapsed'] == merge_all.groupby(['primarymrn','pat_enc_csn_id'])['time_elapsed'].transform(max))\
              & (merge_all.death_flag_inhospital == 1)).astype(float)
merge_all['death_flag_next_period'] = (((merge_all['time_sequence'] == merge_all.groupby(['primarymrn','pat_enc_csn_id'])['time_sequence'].transform(max)-1)\
                                        | (merge_all.groupby(['primarymrn','pat_enc_csn_id'])['time_sequence'].transform(max) == 0))\
                                        & (merge_all.death_flag_inhospital == 1)).astype(float)
merge_all['death_flag_two_periods'] = ((merge_all['time_sequence'] == merge_all.groupby(['primarymrn','pat_enc_csn_id'])['time_sequence'].transform(max)-2)\
                                        & (merge_all.death_flag_inhospital == 1)).astype(float)
merge_all['death_flag_three_periods'] = ((merge_all['time_sequence'] == merge_all.groupby(['primarymrn','pat_enc_csn_id'])['time_sequence'].transform(max)-3)\
                                        & (merge_all.death_flag_inhospital == 1)).astype(float)
merge_all['death_flag_four_periods'] = ((merge_all['time_sequence'] == merge_all.groupby(['primarymrn','pat_enc_csn_id'])['time_sequence'].transform(max)-4)\
                                        & (merge_all.death_flag_inhospital == 1)).astype(float)
#merge_all[merge_all.death_flag_inhospital==1][['pat_enc_csn_id','current_datetime', 'time_elapsed', 'death_flag_inhospital','death_flag_this_period']].sort_values(['pat_enc_csn_id','time_elapsed']).head(55)

In [ ]:
## Replace missing dose_mde with 0
merge_all.rxdosemin.fillna(0,inplace=True)

In [ ]:
## Set multi-level index
merge_all = merge_all.set_index(['primarymrn', 'pat_enc_csn_id', 'time_sequence'])

## Remove duplicates by multi-index
merge_all = merge_all[~merge_all.index.duplicated(keep='first')]

In [ ]:
time_vars = ['albumin', 'alt', 'ast', 'base_exc_art', 'bilirubin_total', 'co2',
       'creatinine', 'crp', 'glucose', 'hco3_art', 'hco3_ven',
       'hemoglobin', 'inr', 'lactate', 'lactate_bld_arterial',
       'platelets', 'poc_inr', 'potassium', 'prealbumin', 'sed_rate',
       'wbc', 'pkmod_r_cpn_glasgow_coma_scale_score', 'pulse',
       'pulse_oximetry', 'respirations', 'temperature', 'systolic',
       'diastolic', 'temperature_max', 'pulse_max', 'wbc_max', 'lactate_max',
       'inr_max', 'creatinine_max', 'ast_max', 'alt_max',
       'bilirubin_total_max', 'systolic_max', 'potassium_max',
       'temperature_min', 'pulse_min', 'pulse_oximetry_min',
       'platelets_min', 'base_exc_art_min', 'albumin_min', 'systolic_min',
       'potassium_min', 'hemoglobin_min', 'pulse_avg',
       'pulse_oximetry_avg', 'systolic_avg']

non_time = list(set(merge_all.columns.values) - set(time_vars) - set(list(merge_all.columns[merge_all.columns.str.endswith('flag')])))

In [ ]:
%%time

## Fill forward (non-time forever and time-based for 2 periods) and backward (non-time-based variables only) to close gaps in between data points -- slow!
## Limit = 2,  otherwise, results in clinical inconsistencies between similar measures (e.g. lactate and base excess)
merge_all[time_vars] = merge_all[time_vars].sort_index().groupby(level=[0,1]).fillna(method='ffill', downcast='infer', limit=2)
merge_all[non_time] = merge_all[non_time].sort_index().groupby(level=[0,1]).fillna(method='ffill', downcast='infer')
merge_all[non_time] = merge_all[non_time].sort_index().groupby(level=[0,1]).fillna(method='bfill', downcast='infer')

# ## Fill forward (all) and backward (non-time-based variables only) to close gaps in between data points -- slow!
# merge_all = merge_all.sort_index().groupby(level=[0,1]).fillna(method='ffill', downcast='infer')
# merge_all[non_time] = merge_all[non_time].sort_index().groupby(level=[0,1]).fillna(method='bfill', downcast='infer')

In [ ]:
# DataFrameToSQL(merge_all, 'merge_all')

In [ ]:
# ## Left join full data set and transfusions on index
# merge_t = pd.merge(merge_all.copy().reset_index(), transfusion_, left_on=['pat_enc_csn_id', 'time_sequence'], right_on=['pat_enc_csn_id', 'time_sequence'], how='left')
# merge_t[include_columns] = merge_t[include_columns].fillna(0)
# merge_all = merge_t.set_index(['primary_mrn', 'pat_enc_csn_id', 'time_sequence'])

In [ ]:
## Calculate shock index, mean blood pressure (map), pmi, bpai, etc.
merge_all.pulse = merge_all.pulse.astype(float)
merge_all.systolic = merge_all.systolic.astype(float)
merge_all.diastolic = merge_all.diastolic.astype(float)

merge_all['map'] = ((merge_all.diastolic * 2 + merge_all.systolic) / 3)
merge_all['minpulse'] = (220 - merge_all.age) - merge_all.pulse
merge_all['pulse_max_index'] = merge_all.pulse / (220 - merge_all.age)
merge_all['blood_pressure_age_index'] = merge_all.systolic / merge_all.age

merge_all['shock_index'] = merge_all.pulse / merge_all.systolic
merge_all['shock_index_modified'] = merge_all.pulse / merge_all.map
merge_all['shock_index_age'] = merge_all.shock_index * merge_all.age
merge_all['shock_index_reverse'] = merge_all.systolic / merge_all.pulse
merge_all['shock_index_reverse_lt1'] = (merge_all['shock_index_reverse'] < 1).astype(int)

In [ ]:
## Year, month, and weekday of arrival
merge_all['arrival_year'] = merge_all.hosp_admsn_time.dt.year
merge_all['arrival_month'] = merge_all.hosp_admsn_time.dt.month
merge_all['arrival_weekday'] = merge_all.hosp_admsn_time.dt.dayofweek

#merge_all['current_year'] = merge_all.current_datetime.dt.year
#merge_all['current_month'] = merge_all.current_datetime.dt.month
#merge_all['current_weekday'] = merge_all.current_datetime.dt.dayofweek

In [ ]:
## Regroup language, race to include only values with more than 200 patients
merge_all.loc[~merge_all.preferredlanguage.isin(['English','Spanish']), 'preferredlanguage'] = 'Other'
merge_all.loc[~merge_all.firstrace.isin(['White', 'Black', 'Asian']), 'firstrace'] = 'Other'

## Remove invalid ISS observations
merge_all.loc[merge_all.iss.isin(['UA', '', '   ']), 'iss'] = None
merge_all.iss = merge_all.iss.astype(float)

## Regroup smoking_status into bigger categories
merge_all.loc[merge_all.smokingstatus.isin(['Current Every Day Smoker', 'Current Some Day Smoker', 'Smoker, Current Status Unknown', 'Heavy Tobacco Smoker', 'Light Tobacco Smoker']), 'smokingstatus'] = 'Current Smoker'
merge_all.loc[merge_all.smokingstatus.isin(['Never Smoker ', 'Passive Smoke Exposure - Never Smoker']), 'smokingstatus'] = 'Never Smoker'
merge_all.loc[~merge_all.smokingstatus.isin(['Current Smoker', 'Never Smoker', 'Former Smoker']), 'smokingstatus'] = 'Unknown'

## Regroup marital status values
merge_all.loc[merge_all.maritalstatus.isin(['Common Law', 'Significant']), 'maritalstatus'] = 'Married'
merge_all.loc[merge_all.maritalstatus.isin(['Unknown', '*Unspecified']), 'maritalstatus'] = 'Other'

## Regroup specialty to include only values with more than 200 patients
merge_all.loc[merge_all.specialty.isin(['RADIOLOGY','GERIATRICS','GENERAL INTERNAL MEDICINE',
                                        '2SS Observation','OBSTETRICS','PULMONARY','DIABETES',
                                        'INTERNAL MEDICINE','ONCOLOGY','PSYCHIATRY','OCCUPATIONAL MEDICINE',
                                        'PHYSICAL THERAPY','GYNECOLOGY','DERMATOLOGY',
                                        'PHYSICAL MEDICINE & REHAB' ]), 'specialty'] = 'OTHER'

## Regroup financial class categories
merge_all.loc[merge_all.primaryfinancialclass.isin(['Tricare', 'Champva', 'Worker\'s Comp']), 'primaryfinancialclass'] = 'Other'
merge_all.loc[merge_all.primaryfinancialclass == 'Blue Shield', 'primaryfinancialclass'] = 'Commercial'

## Regroup mechanism of injury (MOI)
merge_all.moi.fillna('', inplace=True)
merge_all.loc[merge_all['moi'].str.lower().str.contains('gun') | (merge_all['moi'] == 'GSW'), 'moi'] = 'gsw'
merge_all.loc[merge_all['moi'].str.lower().str.contains('fall'), 'moi'] = 'fall'
merge_all.loc[merge_all['moi'].str.lower().str.contains('mva') | (merge_all['moi'] == 'MVC'), 'moi'] = 'mvc'
merge_all.loc[merge_all['moi'].str.lower().str.contains('mtrcycle'), 'moi'] = 'mcc'
merge_all.loc[merge_all['moi'].isin(['Not Applicable                          ', 'Other', '                                        ']),'moi'] = 'other'
merge_all.loc[merge_all['moi'] == 'MPC', 'moi'] = 'mpc'
for x in ['cut', 'stab']: merge_all.loc[merge_all['moi'].str.lower().str.contains(x), 'moi'] = 'sharp'
for x in ['burn', 'hot']: merge_all.loc[merge_all['moi'].str.lower().str.contains(x), 'moi'] = 'burn'
for x in ['assault', 'struck', 'fight']: merge_all.loc[merge_all['moi'].str.lower().str.contains(x), 'moi'] = 'assault'
merge_all.loc[~merge_all['moi'].isin(['fall','mvc','burn','sharp','assault','mcc','gsw','mpc']),'moi'] = 'other'

## Recode age as bins
bins = [0, 15, 24, 44, 64, 120]
group_names = ['<16', '16-24', '25-44', '45-64', '>=65']
merge_all['age_bin'] = pd.cut(merge_all.age, bins = bins, labels = group_names)
merge_all['age_lt16'] = pd.get_dummies(merge_all.age_bin)['<16']
merge_all['age_16-24'] = pd.get_dummies(merge_all.age_bin)['16-24']
merge_all['age_25-44'] = pd.get_dummies(merge_all.age_bin)['25-44']
merge_all['age_45-64'] = pd.get_dummies(merge_all.age_bin)['45-64']
merge_all['age_gt65'] = pd.get_dummies(merge_all.age_bin)['>=65']
#Set all age columns to missing if missing in original
merge_all.loc[merge_all.age_bin.isnull(), merge_all.columns.str.startswith("age_")] = np.nan


In [ ]:
try:
    ## Recode sex, race, financial class
    merge_all['female'] = pd.get_dummies(merge_all, columns=['sex'], drop_first=True)['sex_Female']
    merge_all['hispanic'] = pd.get_dummies(merge_all['ethnicity'])['Hispanic']
    
    ## Drop recoded/replaced/missing variables
    merge_all.drop(['sex', 'ethnicity', 'death_case', 'pkip_r_cam-icu_feature_1', 'pkip_r_cam-icu_feature_2',
                    'pkip_r_cam-icu_feature_3', 'pkip_r_cam-icu_feature_4','pain_scale','pkmod_r_vent_mode'], 
                    axis=1, inplace=True)
except:
    pass

In [ ]:
## Replace any infinite values with NaN
merge_all = merge_all.replace([np.inf, -np.inf], np.nan)

In [ ]:
## Recode all labs and vitals as numeric
time_vars_all = time_vars + ['map', 'minpulse', 'pulse_max_index', 'blood_pressure_age_index', 'shock_index', 'shock_index_modified',
              'shock_index_age', 'shock_index_reverse']

for col in time_vars_all:
    merge_all[col] = pd.to_numeric(merge_all[col].replace(to_replace=r'[<>A-z ]', value='', regex=True), errors='coerce', downcast='float')

In [ ]:
vitals_plus = list(set(time_vars_all) - set(lab))

# Difference by period offset
df = merge_all.copy()[vitals_plus]
df = df.sort_index().groupby(level=[0,1]).diff(periods=1, axis=0)
df = df.add_suffix('_period_diff')
merge_all = pd.merge(merge_all, df, left_index=True, right_index=True, how='left')
merge_all[list(df.columns.values)] = merge_all[list(df.columns.values)].fillna(0)

In [ ]:
#Excluding burn patients per stakeholder inputs since burn patients can have varying results
merge_all = merge_all[~merge_all.moi.isin(['burn'])]

In [ ]:
DataFrameToSQL(merge_all.copy().reset_index(), 'merge_all')

In [ ]:
merge_all = SQLToDataFrame(table_name='merge_all', date_columns=['date_of_arrival', 'current_datetime', 'current_datetime_min', 'birthdate', 'deathdate', 'hosp_admsn_time', 'hosp_dischrg_time'])
# merge_all['time_elapsed'] = merge_all.time_sequence * np.timedelta64(12, 'h')
# merge_all.set_index(['primary_mrn','pat_enc_csn_id','time_sequence'], inplace=True)

## EXPLORATION

In [ ]:
## Pivot tables for demographics
categoricals = ['moi', 'specialty',  'maritalstatus','smokingstatus', 'primaryfinancialclass', 
                'preferredlanguage', 'firstrace', 'arrival_year', 'arrival_month', 'arrival_weekday']
                #'current_year', 'current_month', 'current_weekday']
df = merge_all.copy().reset_index()
df = df[~df.primarymrn.duplicated(keep='first')]
df['i'] = 1

for each in categoricals:
    counts = pd.pivot_table(df, values='i', index=each, columns=['death_flag_inhospital'], aggfunc='count')
    print(counts)    

In [ ]:
data = merge_all.reset_index().set_index('pat_enc_csn_id')
data = data[~data.index.duplicated(keep='last')]

mn.bar(data[time_vars])

In [ ]:
df_corr = merge_all.corr()

plt.figure(figsize=(12,12))
sns.heatmap(df_corr, vmax=.5, square=True)
plt.show()

print(df_corr[['death_flag_this_period']].abs().sort_values(by='death_flag_this_period', ascending=False))

In [ ]:
merge_all.reset_index().groupby('death_flag_next_period').count()

In [ ]:
#Rough deathrate by MRN
print('Death rate by patient: %0.3f' % (merge_all.reset_index()[['pat_enc_csn_id','death_flag_next_period']].drop_duplicates().death_flag_next_period.mean()))

In [ ]:
#Count unique patients by year
from pylab import rcParams
rcParams['figure.figsize'] = 8, 8
t_cohort = merge_all.reset_index()[['date_of_arrival','primarymrn']].drop_duplicates()
t_cohort = t_cohort.set_index('date_of_arrival')
t_cohort = t_cohort.resample('12M').count()#.plot()
ax = t_cohort.plot(kind='area', alpha=.7, title="Patient Count Over Time");

In [ ]:
merge_all.reset_index()[['primarymrn','date_of_arrival','death_flag_inhospital']].drop_duplicates().set_index('date_of_arrival')\
.resample('12M').mean().drop('primarymrn',axis=1).plot.area(stacked=False)

In [ ]:
sns.boxplot(x="age_bin", y="glucose_random", hue="death_flag_inhospital", data=merge_all, palette="PRGn")